### Data Petri Net - Example for Explainable Representations

The used p2p base event log has the following implemented decision points with guards:

- Request Manager or Standard Approval:
    - Request Manager Approval if total_price >= 800
    - Request Standard Approval if total_price <= 1000
- Manager Rejection or Approval:
    - Manager Rejection if total_price >= 400 and random_exponential(100, 200) > 100
    - Otherwise Manager Approval
- Standard Rejection or Approval:
    - Standard Rejection if total_price >= 400 and random_exponential(100, 200) > 100
    - Otherwise Standard Approval

In [1]:
import os
from exdpn.util import import_log
from exdpn.data_petri_net import data_petri_net
from exdpn.guards import ML_Technique
import matplotlib.pyplot as plt 

ImportError: cannot import name 'import_log' from 'exdpn.util' (unknown location)

#### First, create a Data Petri Net.

In [ ]:
event_log_base = import_log(os.path.join(os.getcwd(), "..", 'datasets', 'p2p_base.xes'))
dpn_base = data_petri_net.Data_Petri_Net(event_log = event_log_base, 
                                         event_level_attributes = ['item_category','item_id','item_amount','supplier','total_price'],
                                         verbose = False)

#### Closer look at the decision points.

In [ ]:
decision_points = list(dpn_base.decision_points)
decision_points

#### Select one decision point for the further anaylsis.

In [ ]:
# create dictionary to map place names to actual places
place_names = {dp.name.split(", {")[0]+")": dp.name for dp in decision_points}
place_dict = {name: dp for dp in decision_points for name in place_names if place_names[name] == dp.name} 
# select a certrain decision point and the corresponding data set 
decision_point = place_dict["({'create purchase order'})"]
decision_point

#### Compare Performances of all machine learning techniques.

In [ ]:
comparision_plot = dpn_base.guard_manager_per_place[decision_point].get_comparison_plot()
print("Best guard:", dpn_base.get_guard_at_place(decision_point))

#### Now get explainable representations of all machine learing guards at that decision point.

In [ ]:
dt_guard = dpn_base.guard_manager_per_place[decision_point].guards_list[ML_Technique.DT]
if dt_guard.is_explainable():
    print("Decision Tree")
    dt_explainable_representation = dt_guard.get_explainable_representation()

In [ ]:
svm_guard = dpn_base.guard_manager_per_place[decision_point].guards_list[ML_Technique.SVM]
if svm_guard.is_explainable():
    print("Support Vector Machine")
    svm_guard.get_explainable_representation(dpn_base.guard_manager_per_place[decision_point].X_test)

In [ ]:
nn_guard = dpn_base.guard_manager_per_place[decision_point].guards_list[ML_Technique.NN]
if nn_guard.is_explainable():
    print("Neural Network")
    # use sample of test data to speed up computation of explainable representation
    sampled_test_data = dpn_base.guard_manager_per_place[decision_point].X_test.sample(n = min(100, len(dpn_base.guard_manager_per_place[decision_point].X_test)))
    nn_guard.get_explainable_representation(sampled_test_data)

In [ ]:
lr_guard = dpn_base.guard_manager_per_place[decision_point].guards_list[ML_Technique.LR]
if lr_guard.is_explainable():
    print("Logistic Regression")
    lr_guard.get_explainable_representation(dpn_base.guard_manager_per_place[decision_point].X_test)